# Developer Quickstart

The OpenAI API provides a simple interface to state-of-the-art AI models for text generation, natural language processing, computer vision, and more. This example generates text output from a prompt, as you might using ChatGPT.

In [1]:
# Initalize Openai Client library and Setup API keys
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Initialize the client with API key from environment
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Text Creation

In [2]:
response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under a silver moon, a gentle unicorn named Luna soared over sparkling meadows, whispering sweet dreams to every sleeping flower.


## Analyze Image Inputs

You can provide image inputs to the model as well. Scan receipts, analyze screenshots, or find objects in the real world with computer vision.

In [3]:
response = client.responses.create(
    model="gpt-4.1",
    input=[
        {"role": "user", "content": "What two dogs are in this image?"},
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Comparison_of_a_wolf_and_a_pug.png/1920px-Comparison_of_a_wolf_and_a_pug.png"
                }
            ]
        }
    ]
)

print(response.output_text)

The image shows two animals side by side. On the left is a **wolf**, and on the right is a **pug** (a breed of domestic dog). The wolf is not a domesticated dog breed but is closely related to dogs, while the pug is a popular companion dog.


## Extend the model with tools

Give the model access to new data and capabilities using tools. You can either call your own custom code, or use one of OpenAI's powerful built-in tools. This example uses web search to give the model access to the latest information on the Internet.

In [4]:
response = client.responses.create(
    model="gpt-4.1",
    tools=[{"type": "web_search_preview"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

As of June 24, 2025, here are some recent positive news stories:

**1. 12-Year-Old Girl Plants 150,000 Trees in India**

A young environmentalist in India has achieved a remarkable milestone by planting 150,000 trees, significantly contributing to reforestation efforts and inspiring others to take action against deforestation. ([goodnewsnetwork.org](https://www.goodnewsnetwork.org/?utm_source=openai))

**2. China Achieves 'Excellent' Water Quality in 90% of Rivers and Lakes**

China has reported that 90% of its rivers and lakes now meet 'excellent' water quality standards. This achievement reflects the country's commitment to environmental protection and sustainable development. ([goodnewsnetwork.org](https://www.goodnewsnetwork.org/?utm_source=openai))

**3. Yurok Tribe Celebrates Return of Ancestral Homelands**

The Yurok Tribe in the United States is celebrating the return of their ancestral lands, marking a significant step in restoring Indigenous rights and preserving cultural her

## Deliver blazing fast AI experiences

Using either the new Realtime API or server-sent streaming events, you can build high performance, low-latency experiences for your users.

In [5]:
stream = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": "Say 'double bubble bath'.",
        },
    ],
    stream=True,
)

## Build agents

Use the OpenAI platform to build agents capable of taking action—like controlling computers—on behalf of your users. Use the Agents SDK for Python or TypeScript to create orchestration logic on the backend.

In [6]:
import asyncio
from agents import Agent, Runner

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)

# Define your async function
async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    return result.final_output

# In Jupyter, use await directly
result = await main()
print(result)

¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?
